In [ ]:
#import dependencies

In [ ]:
#call in csv file and set up API keys

In [ ]:
#clean up data if needed

In [ ]:
#display results for microplastics over time

In [ ]:
#display results for cancer rates over time 

In [ ]:
#display regression plots 